# How to get Long/Lat Data from Tweets

-  Data pulled using AppAuth Handler through Tweepy 
-  Search Query "Climate Change"

In [1]:
import pandas as pd 
import json
import numpy as np 

In [2]:
%%bash 
pwd

/Users/sarahalamdari/Desktop/github/DIRECT_capstone/examples


In [3]:
df = pd.read_json("../wyns/data/tweets.txt",  lines=True)  # json file 

In [4]:
#  Check dictionary keys

print (list(df.columns.values))

['contributors', 'coordinates', 'created_at', 'display_text_range', 'entities', 'extended_entities', 'favorite_count', 'favorited', 'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata', 'place', 'possibly_sensitive', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'retweet_count', 'retweeted', 'retweeted_status', 'source', 'truncated', 'user', 'withheld_in_countries']


In [5]:
print ("Corpus contains {0} tweets".format(len(df)))

Corpus contains 89688 tweets


In [32]:
print (df['place'].dropna())
print (df['place'])

127      {'contained_within': [], 'url': 'https://api.t...
191      {'contained_within': [], 'url': 'https://api.t...
370      {'contained_within': [], 'url': 'https://api.t...
574      {'contained_within': [], 'url': 'https://api.t...
589      {'contained_within': [], 'url': 'https://api.t...
612      {'contained_within': [], 'url': 'https://api.t...
632      {'contained_within': [], 'url': 'https://api.t...
712      {'contained_within': [], 'url': 'https://api.t...
927      {'contained_within': [], 'url': 'https://api.t...
986      {'contained_within': [], 'url': 'https://api.t...
1395     {'contained_within': [], 'url': 'https://api.t...
1401     {'contained_within': [], 'url': 'https://api.t...
1441     {'contained_within': [], 'url': 'https://api.t...
1449     {'contained_within': [], 'url': 'https://api.t...
1490     {'contained_within': [], 'url': 'https://api.t...
1541     {'contained_within': [], 'url': 'https://api.t...
1559     {'contained_within': [], 'url': 'https://api.t.

In [6]:
print (df['id'][0])
print (df['full_text'][0])

1007359932944003074
RT @NatGeo: For the Marshall Islands, climate change isn't some distant, future danger: It's already wreaking havoc. https://t.co/juuru9xc61


In [7]:
# Check if Wes' Code catches all tweet text 

full_text = []
i = 0
with open("../wyns/data/tweets.txt") as f:
    for line in f:
        b = json.loads(line)
#         for key in b['retweeted_status'].keys():
#             print(key)
        if 'retweeted_status' in b:
            if 'extended_tweet' in b['retweeted_status']:
              #  print(b['retweeted_status']['extended_tweet']['full_text'])
                full_text.append(b['retweeted_status']['extended_tweet']['full_text'])
                #retweeted status -> usr -> location 
            else:
              #  print(b['text'])
                full_text.append(b['full_text'])
        elif 'extended_tweet' in b:
           # print(b['extended_tweet']['full_text'])
            full_text.append(b['extended_tweet']['full_text'])
        elif 'text' in b:
           # print(b['text'])
            full_text.append(b['full_text'])

        #print(i)
        i += 1
#         break
print('Tweets found: ',len(full_text))

Tweets found:  64312


In [23]:
#Long/Lat Format

#print (df['place'][29]['bounding_box']['coordinates'][0][0][1])
#print (df['place'].dropna())
#print (df['place'][8073])
#print (df['geo'].dropna()[8073]['coordinates'][0])
#print (df['truncated'])

print (df['full_text'][64009])

RT @watspn1013: Scott Pruit, Director of EPA received a 3,000 page paper with 10,000 footnotes providing documentation of his stance that c…


In [8]:
#  Try saving data to 2D array [coordinate, date, full_text]

data = []

with open("../wyns/data/tweets.txt") as f:
    for line in f:
        b = json.loads(line) 
        if b['full_text'] != data:
            if b['geo'] is not None:  # Checks to see if any coord data was saved in geo
                data.append([b['geo']['coordinates'][0],
                             b['geo']['coordinates'][1],
                             b['place']['full_name'],
                             b['created_at'], 
                             b['full_text']])
            elif b['place'] is not None: # Checks to see if any coord data saved in place 
                data.append([b['place']['bounding_box']['coordinates'][0][0][0],
                             b['place']['bounding_box']['coordinates'][0][0][1],
                             b['place']['full_name'],
                             b['created_at'],
                             b['full_text']])
        else:
            pass 

print(np.array(data).shape)
sample = np.array(data)
#print (sample[0:10])

(874, 5)


In [12]:
print (sample[0:10])

[['-73.946553' '41.168161' 'Croton-on-Hudson, NY'
  'Thu Jun 14 20:23:39 +0000 2018'
  'As a pundit once said: “Gravity isn’t just a suggestion, it’s the law.”\n\nGovernment can try to make it illegal for staff to discuss climate change - but natural laws don’t pay attention to executive orders - or to the profits of fossil fuel companies.\n\n@NickyTheKat @BXEAction https://t.co/Z7mKbsjTIw']
 ['-74.026675' '40.683935' 'Manhattan, NY' 'Thu Jun 14 20:19:59 +0000 2018'
  'Thanks for raising this topic for attention and helping us to include it in our climate change agendas. https://t.co/9JINovMthk']
 ['10.5922629' '55.1365705' 'Sweden' 'Thu Jun 14 20:09:43 +0000 2018'
  'We’re simulation the onslaught of climate change in subarctic tundra with litter and nutient additions. Year 2 applications are now administered #soilmicrobes #climatescience #microbialecology #primingeffect #fieldwork #Abisko https://t.co/OTa41puRqG']
 ['-79.509317' '44.016826' 'Newmarket, Ontario'
  'Thu Jun 14 19:57:14

In [ ]:
sample_df = pd.DataFrame(sample, columns=['long','lat','self_reported_location','date','text'])
sample_df.to_csv('../wyns/data/sample.csv', encoding='utf-8', index=False)
sample_df.head()

In [ ]:
with open('sample.csv') as f: 
    for line in f:
        print (f)

In [ ]:
def check_tweet_data(tweet):
    data = []
    if tweet['geo'] is not None:  # Checks to see if any coord data was saved in geo
        data.append([tweet['geo']['coordinates'][0],
                     tweet['geo']['coordinates'][1],
                     tweet['place']['full_name'],
                     tweet['created_at'], 
                     tweet['text']])
    elif tweet['place'] is not None: # Checks to see if any coord data saved in place 
        data.append([tweet['place']['bounding_box']['coordinates'][0][0][0],
                     tweet['place']['bounding_box']['coordinates'][0][0][1],
                     tweet['place']['full_name'],
                     tweet['created_at'],
                     tweet['text']])
    else:
        pass 
    sample = np.array(data)
    sample_df = pd.DataFrame(sample, columns=['long','lat','self_reported_location','date','text'])
    

In [ ]:
def check_if_exists(file_name):
    
    
    with open(file_name, 'a') as f:
    df.to_csv(f, header=False)
